In [1]:
import copy
import os
import numpy as np
import pandas as pd

In [2]:
pattern_date = '2023-12-24'
root_dir = f'{"/".join(os.getcwd().split("/")[:-1])}'
format_anno_dir = f'{root_dir}/data/outputs/01.format_and_cv'
pose_output_dir = f'{root_dir}/data/outputs/02.pose_estimation'
result_dir = f'{root_dir}/data/outputs/04.highplace_detection/00.person_detection_add_label'

In [3]:
annotation_person_df = pd.read_csv(f'{pose_output_dir}/person_eval_annotation-{pattern_date}.csv')
pred_person_df = pd.read_csv(f'{pose_output_dir}/person_eval_pred-{pattern_date}.csv')

## 人検出した切り取り画像に、ラベルを付与

In [4]:
def calculate_intersection_per_label(pred_bbox, label_bbox):
    # バウンディングボックスの座標情報を取得
    x1_1, y1_1, x2_1, y2_1 = label_bbox
    x1_2, y1_2, x2_2, y2_2 = pred_bbox
    
    # 交差する領域を計算
    x_intersection = max(0, min(x2_1, x2_2) - max(x1_1, x1_2))
    y_intersection = max(0, min(y2_1, y2_2) - max(y1_1, y1_2))
    intersection_area = x_intersection * y_intersection

    # 正解ラベルの面積
    area_label_bbox = (x2_1 - x1_1) * (y2_1 - y1_1)

    # 交差する領域の面積 / 正解ラベルの面積
    return intersection_area / area_label_bbox

In [5]:
def calculate_iou(box1, box2):
    # バウンディングボックスの座標情報を取得
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # 交差する領域を計算
    intersection_x = max(0, min(x1 + w1, x2 + w2) - max(x1, x2))
    intersection_y = max(0, min(y1 + h1, y2 + h2) - max(y1, y2))
    
    # 交差領域の面積と各バウンディングボックスの面積を計算
    intersection_area = intersection_x * intersection_y
    box1_area = w1 * h1
    box2_area = w2 * h2
    
    # IoUを計算
    iou = intersection_area / (box1_area + box2_area - intersection_area)
    
    return iou

In [6]:
def make_person_cls(row, ratio_threshold=0.8, iou_threshold=0.3):
    img_key = row["unique_key"]
    pred_bbox = [row["left"], row["top"], row["right"], row["bottom"]]
    _annotation_person_df = copy.copy(annotation_person_df[annotation_person_df["unique_key"] == img_key])
    for gt_row in _annotation_person_df.values:
        if (
            calculate_intersection_per_label(pred_bbox, gt_row[3:7]) >= ratio_threshold
        ) and (
            calculate_iou(pred_bbox, gt_row[3:7]) >= iou_threshold
        ):
            return gt_row[2]
    return "detection-miss"

In [7]:
pred_person_df["label"] = pred_person_df.apply(make_person_cls, axis=1)

In [8]:
print('学習データ内の評価用ピックアップデータのラベル件数')
display(pd.DataFrame(pred_person_df[
    (pred_person_df['unique_key'].str.contains('fixed-point-camera')) | 
    (pred_person_df['unique_key'].str.contains('for-learning/2023-11-19-omaezaki-500')) | 
    (pred_person_df['unique_key'].str.contains('for-learning/2023-11-23-mie-safetybelt'))
]['label'].value_counts()))

print('評価用データのラベル件数')
display(pd.DataFrame(pred_person_df[
    (pred_person_df['validation'] == 999)
]['label'].value_counts()))

学習データ内の評価用ピックアップデータのラベル件数


,count
label,
person,1731
detection-miss,193
person in high place,81


評価用データのラベル件数


,count
label,
person,50
person in high place,36
detection-miss,16


In [9]:
os.makedirs(result_dir, exist_ok=True)
pred_person_df.to_csv(f'{result_dir}/person_eval_pred_add_label-{pattern_date}.csv', index=False)